# Building a circuit : Introductions to  Operations, Circuit, CircuitDAG, DAG visualization, openQASM/qiskit visualization, and pre-built circuits

## Overview [Public API]

The circuit classes contain our quantum algorithms (that is, the time-ordered sequence of operations which we apply on specific qubits to get from our input states to our target states).

They can, in principle, be represented in multiple different ways. Currently, our code base supports:
1. Directed-acyclic graph (DAG) circuit representation, for simulation
2. openQASM representation, for saving data, conversion to IBM's qiskit platform (note: this is only a visual compatibility at this point--more work is likely required to be able to simulate our circuits in qiskit).

## Photonic Circuits [Public API]

Our discrete photonic quantum circuits have 3 types of registers:
1. **Classical registers:** these registers store classical information like measurement results and can be used to control the application of gates on qubits. Typically, only one classical register is needed
2. **Photonic quantum registers:** these qubit registers are the qubits used in the construction of our target quantum state.
3. **Emitter quantum registers:** these registers are solid state qubits, which can generate photons (while this is not enforced by the circuit class, the emitters in fact generate the photon).

All the registers are zero-indexed. At this stage in the implementation, we only consider single-qubit registers.

## Operations

**[Public API]**

We support a set of operations which can be sequentially applied to form a quantum circuit. Operations are defined by the following:

1. **Operation type:** this is the class of the Operation object, and tells us what type of gate the Operation applies (e.g. CNOT, Hadamard)
2. **Their operation registers:** The operation registers defines the qubit/classical bit used by the operation. This must specify both the type of the register (classical, photonic quantum, emitter quantum), and its index.

In [11]:
""" Operation Examples """
import src.ops as ops

# Single Qubit Examples
# Hadamard on emitter 0
h = ops.Hadamard(register=0, reg_type='e')
# Pauli X gate on photon 1
paulix = ops.SigmaX(register=1, reg_type='p')


# Controlled Pair Example
# CNOT controlled by emitter 0, targeting photon 1
cnot = ops.CNOT(control=0, control_type='e', target=1, target_type='p')


# CLassically controlled pair example
# Classical CNOT, where emitter 2 is measured, saved to classical register 0, 
# and conditionally applied to photon 1
ccnot = ops.ClassicalCNOT(control=2, control_type='e', target=1, target_type='p', c_register=0)

# Classically controlled CNOT + measured qubit reset
# Emitter 0 is measured, saved to classical reg 0, conditional gate on photon 0
ccnot_reset = ops.MeasurementCNOTandReset(control=0, control_type='e',
                                          target=0, target_type='p',
                                          c_register=0)

In certain conditions, it may be useful to define a single operation object which applies multiple gates sequentially.

**NOTE:** this feature is only supported for single-qubit operations at the moment

In [12]:
""" Operation Wrapper Example """

HP_gate = ops.OneQubitGateWrapper([ops.Hadamard, ops.Phase], register=0, reg_type='p')

# NOTE: this applies first a phase gate, then a Hadamard gate on the register (following usual convention)

Optionally, a noise model can be proposed for each `Operation` (by default, the noise model is `NoNoise`).

In [10]:
import src.noise.noise_models as nm
h_with_noise = ops.CNOT(
    control=0,
    control_type='e',
    target=0,
    target_type='p',
    noise = nm.PauliError("X")
)

### Operation Base Classes/Inheritance [Implementer Info]

Because we want to be able to support different state representations with few changes to the API, these operation objects **do not encode how we apply the operation to the state**. That is, there is no code in these Operation objects that tells us how they modify states (this is handled in the compilers).

Then, there is a lot of similarities between the implementation of these different operation objects. To avoid rewriting the code repeatedly (bad practice, since it means each change must be made in multiple places!), we use **base classes** from which specific Operation objects (e.g. the Hadamard and CNOT viewed above) inherit code.

#### OperationBase

All Operation classes **inherit** from `OperationBase`.

**Python note:** A child class (e.g. `Hadamard`) may inherit from a parent class (e.g. `OperationBase`). This means that you can call functions on a child which ARE NOT EXPLICITLY defined in the child, if they are explicitly defined in the parent--if the child does not have its own version of the function, the code called will be the parent-code.

From an implementation point of view, this is also useful in the circuit implementation because the common superclass of the operations allows us to manipulate `Operation` objects in the same way regardless of their actual corresponding gate. 

Operation base has `q_registers`, `q_registers_type` and `c_registers` , in which it saves the information of which quantum/classical registers of the circuit it must act on. It also contains a noise model.

In [13]:
""" A look into OperationBase """

# Operation object which acts on quantum registers 0, 1 and classical register 0
base0 = ops.OperationBase(q_registers=(0, 1), 
                          q_registers_type=('e', 'p'),
                          c_registers=(0,))

# Careful, base0 may not be the same as base1! Order of register matters
base1 = ops.OperationBase(q_registers=(1, 0), 
                          q_registers_type=('e', 'p'),
                          c_registers=(1,))

# NOTE: the comma in c_registers=(0,) is necessary in python to make c_registers a TUPLE (iterable data) instead of an int

#### InputOutputOperationBase, SingleQubitOperationBase, InputOutputOperationBase, ControlledPairOperationBase, ClassicalControlledPairOperationBase

All the above inherit from `OperationBase`: like `OperationBase`, they exist such that we can recycle code between similar operations (for example, all single-qubit unitary gates acting on qubits need similar info stored)

See `ops.py` for their precise implementations

#### The "real" operations

Finally, we get to the "real" operations which we want in our circuit (note that you can PLACE the base classes in the circuit object--mostly for testing purposes--but that the compiler will raise an error if you try to compile the base classes)

These include:

**InputOutputOperationBase** types: Input, Output (these are simply placeholder Operations to signal the beginning/end of the circuit)

**SingleQubitOperationBase** types: Hadamard, SigmaX,SigmaY, SigmaZ

**ControlledPairOperationBase** types: CNOT, CPHASE

**ClassicalControlledPairOperationBase** types: classical CNOT/CPHASE, MeasurementCNOTandReset (same as classical CNOT, with the measured qubit being reset to |0>)

**OperationBase** type (though we may want another base class later for this, if we implement more): MeasurementZ


## Circuit Class

//TODO